In [ ]:
! pip install -q flash-attn --no-build-isolation

In [ ]:
! pip install -q -U transformers accelerate

In [ ]:
! pip install -q transformers datasets peft git+https://github.com/huggingface/trl.git accelerate bitsandbytes packaging ninja sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
drive.mount("/content/drive")
transformer_path = "/content/drive/My Drive/transformers/"

Mounted at /content/drive


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import random
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import numpy as np
import pandas as pd
import transformers
from accelerate import Accelerator
import bitsandbytes as bnb
from datasets import load_dataset, concatenate_datasets

In [ ]:
# HuggingFace Token
token='' # Add your token here

In [ ]:
# Load the dataset
dataset = load_dataset(
    'Amod/mental_health_counseling_conversations',
    token=token,
    cache_dir=transformer_path
)

In [ ]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16 # A100
)

model_name = "meta-llama/Llama-2-7b-chat-hf"

#Load Tokenizer
tokenizer= AutoTokenizer.from_pretrained(
    model_name,
    token=token,
    cache_dir=transformer_path,
)

# Add Padding Token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load the model in 4-bit
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token,
    device_map={"": Accelerator().local_process_index},
    quantization_config=nf4_config,
    attn_implementation="flash_attention_2",
    cache_dir=transformer_path,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

In [ ]:
# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
args = TrainingArguments(
    output_dir=os.path.join(transformer_path, 'result'),
    overwrite_output_dir='True',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=1.0e-4,
    tf32=True, #A100
    max_grad_norm=0.3,
    warmup_ratio=0.3,
    lr_scheduler_type="cosine",
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    push_to_hub=True,
    hub_model_id="clement-cvll/mental-health-llama2",
    hub_token=token,
)

In [ ]:
# System message to better instruct chatbot
system_message = """You are a helpful and and truthful psychology and psychotherapy assistant. Your primary role is to provide empathetic, understanding, and non-judgmental responses to users seeking emotional and psychological support.
                  Always respond with empathy and demonstrate active listening; try to focus on the user. Your responses should reflect that you understand the user's feelings and concerns. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety.
                  Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate.  You are not a licensed medical professional. Do not diagnose or prescribe treatments.
                  Instead, encourage users to consult with a licensed therapist or medical professional for specific advice. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share and reflect.
                  Remember, your goal is to provide a supportive and understanding environment for users to share their feelings and concerns. Always prioritize their well-being and safety."""

def format(entry):
  formatted = f"<s>[INST] <<SYS>>{system_message}<</SYS>>{entry['Context']} [/INST]  {entry['Response']}  </s>"

  return formatted

In [ ]:
train = dataset['train'].select(range(3000))
val = dataset['train'].select(range(3000, 3512))

In [ ]:
print(format(dataset['train'][0]))

<s>[INST] <<SYS>>You are a helpful and and truthful psychology and psychotherapy assistant. Your primary role is to provide empathetic, understanding, and non-judgmental responses to users seeking emotional and psychological support.
                  Always respond with empathy and demonstrate active listening; try to focus on the user. Your responses should reflect that you understand the user's feelings and concerns. If a user expresses thoughts of self-harm, suicide, or harm to others, prioritize their safety.
                  Encourage them to seek immediate professional help and provide emergency contact numbers when appropriate.  You are not a licensed medical professional. Do not diagnose or prescribe treatments.
                  Instead, encourage users to consult with a licensed therapist or medical professional for specific advice. Avoid taking sides or expressing personal opinions. Your role is to provide a safe space for users to share and reflect.
                  Reme

In [ ]:
max_seq_length = 1024 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format,
    neftune_noise_alpha=5,
    args=args,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:235: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import gc

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# train
trainer.train(resume_from_checkpoint=False)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


Epoch,Training Loss,Validation Loss
1,1.110400,1.214333


Checkpoint destination directory /content/drive/My Drive/transformers/result/checkpoint-424 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=424, training_loss=1.4559188293960859, metrics={'train_runtime': 929.3053, 'train_samples_per_second': 1.822, 'train_steps_per_second': 0.456, 'total_flos': 6.907724018850202e+16, 'train_loss': 1.4559188293960859, 'epoch': 1.0})

In [ ]:
# save model
trainer.model.save_pretrained(os.path.join(transformer_path, 'result'))